In [6]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
import pymongo
from collections import defaultdict

In [ ]:
from IPython.display import clear_output

In [28]:
from pubcrawler.article import Article

In [8]:
client = pymongo.MongoClient("localhost", 27017)
articles = client.pmc.articles

In [11]:
# Make sure we have a text index
articles.create_index([("extracted_text", pymongo.TEXT,)])

'extracted_text_text'

In [51]:
import time
from statistics import mean
from IPython.display import clear_output
class Reporter:
    def __init__(self, interval, total):
        self.start = time.time()
        self.this_time = self.start
        self.interval = interval
        self.total = total
        self.times_per_batch = []
    
    def report(self, idx):
        self.last_time = self.this_time
        self.this_time = time.time()
        time_per = (self.this_time - self.last_time) / self.interval
        self.times_per_batch.append(time_per)
        est_time_left = (self.total - idx) * mean(self.times_per_batch[-10:])
        elapsed = time.time() - self.start

        output = "Processed {0} articles ({1:.1f}%) in {2:.0f}m{3:.0f}s; about {4:.0f}m{5:.0f}s left.".format(
            idx,
            idx/self.total * 100,
            elapsed // 60,
            elapsed % 60,
            est_time_left // 60,
            est_time_left % 60)
        clear_output(wait=True)
        print(output)

## Search for articles matching terms

In [12]:
terms = [
    '"field work"',
    '"field study"',
    '"study site"'
]

In [13]:
matching_ids = {}
for term in terms:
    results = articles.find(
        { '$text': { '$search': term } },
        { '_id': '_id', 'score': { '$meta': "textScore" } }
    )
    result_set = {result["_id"] for result in results}
    matching_ids[term] = result_set

In [15]:
result_set

{'1281290',
 '1475787',
 '1832199',
 '1887535',
 '1975471',
 '2231342',
 '2525495',
 '2577634',
 '2602971',
 '2606649',
 '2680901',
 '2744769',
 '2761779',
 '2859400',
 '2860981',
 '2865642',
 '2894007',
 '2914751',
 '2948002',
 '2955963',
 '2965904',
 '3003275',
 '3007589',
 '3011831',
 '3020222',
 '3063209',
 '3068257',
 '3082239',
 '3083404',
 '3083659',
 '3128412',
 '3141142',
 '3154270',
 '3156131',
 '3162569',
 '3163582',
 '3166751',
 '3170595',
 '3196682',
 '3204773',
 '3226633',
 '3228940',
 '3228981',
 '3244420',
 '3256173',
 '3275991',
 '3316559',
 '3342226',
 '3348038',
 '3348575',
 '3364994',
 '3365007',
 '3374614',
 '3376113',
 '3386916',
 '3407111',
 '3416811',
 '3446913',
 '3460928',
 '3467401',
 '3468748',
 '3470929',
 '3473966',
 '3476896',
 '3542179',
 '3570525',
 '3575393',
 '3598340',
 '3642362',
 '3646215',
 '3648503',
 '3651385',
 '3652831',
 '3704955',
 '3708944',
 '3723670',
 '3734204',
 '3740714',
 '3748518',
 '3772094',
 '3776764',
 '3786079',
 '3807633',
 '38

## Count article types and other metadata

TODO: Check if an article's `_id` is in `result_set` and add those to a different default_dict

In [ ]:
id_types = defaultdict(int)
article_types = defaultdict(int)
cursor = articles.find()
reporter = Reporter(50, articles.count_documents({}))
for idx, doc in enumerate(cursor):
    if idx is not 0 and idx % reporter.interval is 0:
        reporter.report(idx)
    article = Article(doc["xml"])
    
    ids = article.pub_ids().keys()
    if len(ids) is 0:
        id_types["none"] += 1
    else:
        for item in ids:
            id_types[item] += 1
    
    article_type = article.article_type()
    if article_type:
        article_types[article_type] += 1

clear_output()

Processed 2300 articles (11.5%) in 1m32s; about 11m11s left.
